## Necessary imports

In [16]:
import os, re # finding all graph files
import numpy as np

# import glob
# import matplotlib.pyplot as plt
# import pandas as pd
# import seaborn as sns
# import pickle
# from vis_utils import load_volume, VolumeVisualizer, ColorMapVisualizer
# from scipy.ndimage import zoom
# from scipy import signal
# from PIL import Image
# from queue import PriorityQueue
# from skimage.morphology import skeletonize_3d, binary_dilation
# from skimage import measure, segmentation, feature
# from skimage.filters import frangi, sato
# from skimage.draw import line_nd
# from skimage import filters, morphology

### All avalible graphs for kidneys

In [17]:
pkl_re = re.compile('.*P[0-9]*\\\\dag.pkl')
data_files = [os.path.join(path, name) for path, _, files in os.walk('.\\data') for name in files]
graph_files = list(filter(pkl_re.match, data_files)) 
print(f"Currently avalible graph files: {graph_files}")

Currently avalible graph files: ['.\\data\\P01\\dag.pkl', '.\\data\\P07\\dag.pkl', '.\\data\\P09\\dag.pkl', '.\\data\\P12\\dag.pkl', '.\\data\\P18\\dag.pkl']


## Graph definition

In [18]:
def get_nodes_with_dfs(root):
    nodes = [root]
    for e in root.edges:
        if e.node_a != root:
            print(e)
        
        nodes += get_nodes_with_dfs(e.node_b)
        
    return nodes


def get_edges_with_dfs(root):
    edges = []
    for e in root.edges:
        edges += [e]
        edges += get_edges_with_dfs(e.node_b)
        
    return edges

In [19]:
class Node:
    def __init__(self, coords):
        self.coords = coords
        self.edges = []
        self.data = {}
            
    def add_edge(self, edge):
        self.edges.append(edge)
        
    def get_neighbours(self):
        return [e.node_a if e.node_a.coords != self.coords else e.node_b for e in self.edges]
    
    def copy_without_edges(self):
        copied_node = Node(self.coords)
        copied_node.data = self.data
        return copied_node
    
    def __setitem__(self, key, value):
        self.data[key] = value
    
    def __getitem__(self, key):
        return self.data[key]
    
    def __hash__(self):
        return hash(self.coords)
    
    def __repr__(self):
        return f'Node {str(self.coords)}'
       
        
class Edge:
    def __init__(self, node_a, node_b):
        self.node_a = node_a
        self.node_b = node_b
        self.data = {}
        
    def __setitem__(self, key, value):
        self.data[key] = value
    
    def __getitem__(self, key):
        return self.data[key]
    
    def __repr__(self):
        return f'Edge {self.node_a.coords} -> {self.node_b.coords}'

    
class DAG:
    def __init__(self, root, volume_shape):
        self.root = root
        self.nodes = get_nodes_with_dfs(root)
        self.edges = get_edges_with_dfs(root)
        self.volume_shape = volume_shape
        self.data = {}
    
    def __setitem__(self, key, value):
        self.data[key] = value
    
    def __getitem__(self, key):
        return self.data[key]
           

def save_dag(dag, filename):
    with open(filename, 'wb') as output:
        pickle.dump(dag, output)
        

def load_dag(filename):
    with open(filename, 'rb') as input_:
        dag = pickle.load(input_)
        return dag

### Example DAG

In [20]:
dag = load_dag(graph_files[0])
# reconstruction = np.load(source_dir + TREE_NAME + '/reconstruction.npy')

## Mean length (L)

In [21]:
def l_ratio(edges, listt):
    print(len(edges), len(listt))
#     print(len(listt))
#     print(np.array([node.edges for node in nodes]).flatten())
    nodes = [edge.node_b for edge in edges]
    if len(nodes) == 0:
        return listt
    
    new_edges = []
    for edge in edges:
        children = edge.node_b.edges
        for e in children:
            listt.append( e['length'] / edge['length'] )
            new_edges.append(e)
    return l_ratio(new_edges, listt)

## Mean diameter (D)

In [29]:
def d_ratio(edges, listt):
    print(len(edges), len(listt))
#     print(len(listt))
#     print(np.array([node.edges for node in nodes]).flatten())
    nodes = [edge.node_b for edge in edges]
    if len(nodes) == 0:
        return listt
    
    new_edges = []
    for edge in edges:
        children = edge.node_b.edges
        for e in children:
            listt.append( e['mean_radius'] / edge['mean_radius'] )
            new_edges.append(e)
    return l_ratio(new_edges, listt)

## Calculating parameters for graphs

In [28]:
dag = load_dag(graph_files[1])

xd = l_ratio(dag.root.edges, [])
np.mean(xd[2:])

1 0
2 2
4 6
8 14
16 30
32 62
58 120
74 194
105 299
141 440
201 641
257 898
300 1198
341 1539
345 1884
385 2269
405 2674
369 3043
385 3428
354 3782
311 4093
230 4323
165 4488
111 4599
71 4670
40 4710
14 4724
5 4729
2 4731
2 4733
0 4733


1.7432396478856604

In [30]:
xd2 = d_ratio(dag.root.edges, [])

1 0
2 2
4 6
8 14
16 30
32 62
58 120
74 194
105 299
141 440
201 641
257 898
300 1198
341 1539
345 1884
385 2269
405 2674
369 3043
385 3428
354 3782
311 4093
230 4323
165 4488
111 4599
71 4670
40 4710
14 4724
5 4729
2 4731
2 4733
0 4733


In [31]:
np.mean(xd2)

1.742845955921278